In [ ]:
### Geenerate a summary table, agg on time, for all items

In [1]:
import pandas as pd

# Load the CSV file
data_file = "/home/py/data/PREDICT_SALES_v1/data/processed/merged.csv"
df = pd.read_csv(data_file)

# Rename columns to match your requirements
df.rename(columns={
    'predict_cluster_kmeans_fs_sales': 'kmeans',
    'predict_cluster_volume_fs_sales': 'volume',
    'predict_cluster_seasonal_fs_sales': 'seasonal',
    'predict_cluster_hac_s_sales': 'hac',
    'predict_all_sales': 'all'
}, inplace=True)

# Specify your start week here
start_week = 2839  # for example

# Filter the DataFrame for rows where epoch_week is greater or equal to start_week
filtered_df = df[df['epoch_week'] >= start_week]

# Define the columns to sum
columns_to_sum = ['sales', 'ly', 'kmeans', 'volume', 'seasonal', 'hac', 'all', 'yhat']

# Sum the columns for each item_id
sum_df = filtered_df.groupby('item_id')[columns_to_sum].sum().reset_index()

# Calculate absolute differences and find closest value
model_cols = ['kmeans', 'volume', 'seasonal', 'hac', 'all']
columns_to_sum = ['sales', 'ly', 'kmeans', 'volume', 'seasonal', 'hac', 'all', 'yhat', 'lgb']
# sum_df['lgb'] = sum_df[model_cols].sub(sum_df['sales'], axis=0).abs().idxmin(axis=1).apply(lambda x: sum_df.loc[sum_df.name, x], axis=1)
closest_columns = sum_df[model_cols].sub(sum_df['sales'], axis=0).abs().idxmin(axis=1)

# Extract values from closest columns using list comprehension
sum_df['lgb'] = [sum_df.loc[idx, col] for idx, col in closest_columns.items()]
# Convert the 'lgb' column to integer type
sum_df['lgb'] = sum_df['lgb'].astype(int)

# Calculate the differences
for col in columns_to_sum[1:]:  # skip 'sales' since it's the base for subtraction
    sum_df[f'sales_diff_{col}'] = sum_df['sales'] - sum_df[col]

# # Rename columns to match your requirements
# sum_df.rename(columns={
#     'predict_cluster_kmeans_fs_sales': 'sum_kmeans',
#     'predict_cluster_volume_fs_sales': 'sum_volume',
#     'predict_cluster_seasonal_fs_sales': 'sum_seasonal',
#     'predict_cluster_hac_s_sales': 'sum_hac',
#     'predict_all_sales': 'sum_all',
#     'yhat': 'sum_yhat'
# }, inplace=True)
# 'item_id', 'sales', 'ly', 'kmeans', 'volume', 'seasonal', 'hac', 'all',
#        'yhat', 'sales_diff_ly', 'sales_diff_kmeans', 'sales_diff_volume',
#        'sales_diff_season
# Select and reorder columns as necessary
final_columns = [
    'item_id', 'sales', 'ly', 'lgb',
    'yhat', 'sales_diff_ly', 'sales_diff_lgb','sales_diff_yhat'
]
final_df = sum_df[final_columns]

# Display the final DataFrame
print(final_df.head())
data_file = "/home/py/data/PREDICT_SALES_v1/data/processed/summary_table.csv"
final_df.to_csv(data_file, index=False)

   item_id    sales         ly      lgb      yhat  sales_diff_ly  \
0  NOISE20   457584   611664.0   513850  456454.0      -154080.0   
1  NOISE21  1001938  1057665.0  1051598  931509.0       -55727.0   
2  NOISE22    76716   116942.0    95806  109599.0       -40226.0   
3  NOISE23   415338   632826.0   541016  598480.0      -217488.0   
4  NOISE24  1115576  1307600.0  1156332  994942.0      -192024.0   

   sales_diff_lgb  sales_diff_yhat  
0          -56266           1130.0  
1          -49660          70429.0  
2          -19090         -32883.0  
3         -125678        -183142.0  
4          -40756         120634.0  


In [4]:
print(sum_df.columns)

Index(['item_id', 'sales', 'ly', 'kmeans', 'volume', 'seasonal', 'hac', 'all',
       'yhat', 'lgb', 'sales_diff_ly', 'sales_diff_kmeans',
       'sales_diff_volume', 'sales_diff_seasonal', 'sales_diff_hac',
       'sales_diff_all', 'sales_diff_yhat', 'sales_diff_lgb'],
      dtype='object')


## Generate summary stats/performance

In [6]:
import pandas as pd
import numpy as np

# Example data structure (columns: Item, Sales, LY, Prophet, LightGBM, Time)
data_file = "/home/py/data/PREDICT_SALES_v1/data/processed/summary_table.csv"
df = pd.read_csv(data_file)

# Summary statistics for each item
item_stats = df.groupby("item_id").agg({
    "sales": ["mean", "std", "min", "max"],
    "ly": lambda x: np.sqrt(np.mean((x - df["sales"])**2)),  # RMSE for LY model
    "yhat": lambda x: np.mean(np.abs((x - df["sales"])/df["sales"]))*100,  # MAPE
    "lgb": lambda x: np.mean(x - df["sales"])  # Mean Bias
}).round(2)

print(item_stats)

                  sales                              ly     yhat       lgb
                   mean std      min      max  <lambda> <lambda>  <lambda>
item_id                                                                   
NOISE20        457584.0 NaN   457584   457584  154080.0     0.25   56266.0
NOISE21       1001938.0 NaN  1001938  1001938   55727.0     7.03   49660.0
NOISE22         76716.0 NaN    76716    76716   40226.0    42.86   19090.0
NOISE23        415338.0 NaN   415338   415338  217488.0    44.09  125678.0
NOISE24       1115576.0 NaN  1115576  1115576  192024.0    10.81   40756.0
...                 ...  ..      ...      ...       ...      ...       ...
STYCOL006692    27156.0 NaN    27156    27156   24671.0    91.45  -20130.0
STYCOL006693    18927.0 NaN    18927    18927   16241.0    86.62  -10448.0
STYCOL006694    13246.0 NaN    13246    13246   10809.0    83.13     411.0
STYCOL006695    13108.0 NaN    13108    13108   10797.0    83.99   -3860.0
STYCOL006696    23436.0 N